In [1]:
import TradeClient
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
ins_futures= TradeClient.FuturesOrder()

In [7]:
data = asyncio.run(ins_futures.get_account_balance())

In [8]:
len(data)

2

In [6]:
utils._convert_to_datetime(1736559985813)

'2025-01-11 10:46:25'

In [9]:
data

{'XRPUSDT': {'initialMargin': 5.15636,
  'maintMargin': 0.0257818,
  'unrealizedProfit': 0.07546,
  'positionInitialMargin': 5.15636,
  'openOrderInitialMargin': 0,
  'leverage': 1,
  'isolated': True,
  'entryPrice': 2.3095,
  'breakEvenPrice': 2.31065475,
  'maxNotional': 100000000.0,
  'positionSide': 'BOTH',
  'positionAmt': 2.2,
  'notional': 5.15636,
  'isolatedWallet': 5.08147368,
  'updateTime': 1736553600616,
  'bidNotional': 0,
  'askNotional': 0},
 'STEEMUSDT': {'initialMargin': 0.28021005,
  'maintMargin': 0.00420315,
  'unrealizedProfit': -0.00592005,
  'positionInitialMargin': 0.28021005,
  'openOrderInitialMargin': 0,
  'leverage': 1,
  'isolated': True,
  'entryPrice': 0.27429,
  'breakEvenPrice': 0.136489305,
  'maxNotional': 2000000.0,
  'positionSide': 'BOTH',
  'positionAmt': -1,
  'notional': -0.28021005,
  'isolatedWallet': 0.27281394,
  'updateTime': 1736559985813,
  'bidNotional': 0,
  'askNotional': 0}}

In [10]:
data2 = asyncio.run(ins_futures.get_account_balance())

In [20]:
for i, a in data.items():
    print(i)

XRPUSDT
STEEMUSDT


In [18]:
if not data2:
    print(True)

True


In [19]:
if data:
    print(True)

True


In [25]:
    # klline 매개변수에 들어갈 유효 limit
    def __get_valid_kline_limit(self, interval: str) -> Dict:
        # 상수 설정
        VALID_INTERVAL_UNITS = ["m", "h", "d", "w", "M"]
        MINUTES_PER_DAY = 1_440
        MAX_KLINE_LIMIT = 1_000

        # interval 유효성 검사
        interval_unit = interval[-1]
        interval_value = int(interval[:-1])

        if interval not in self.KLINE_INTERVALS:
            raise ValueError(f"유효하지 않은 interval입니다: {interval}")

        # 분 단위로 interval 변환
        interval_minutes = interval_value * {"m": 1, "h": 60, "d": 1440}[interval_unit]

        # 최대 수집 가능 일 수 및 제한 계산
        intervals_per_day = MINUTES_PER_DAY / interval_minutes
        max_days = int(MAX_KLINE_LIMIT / intervals_per_day)
        max_limit = int(intervals_per_day * max_days)

        # 결과 반환
        return (max_days, max_limit)

    # kline 매개변수에 들어가는 limit을 day기준으로 개수를 계산한다.
    def __get_kline_limit_by_days(self, interval: str, days: int) -> int:
        """
        1. 기능 : kline 매개변수에 들어갈 day 기간 만큼 interval의 limit값을 구한다.
        2. 매개변수
            1) interval : KLINE_INTERVALS 속성 참조
            2) day : 기간을 정한다.
        """
        day, limit = self.__get_valid_kline_limit(interval=interval)
        if day == 0:
            return 1000
        max_klines_per_day = (limit / day) * days

        if max_klines_per_day > 1_000:
            print(
                f"The calculated limit exceeds the maximum allowed value: {max_klines_per_day}. Limiting to 1,000."
            )
            return 1_000

        return int(max_klines_per_day)

    # kline전체를 days기준 범위로 업데이트한다.
    async def update_all_klines(self, days: int = 1):
        """
        1. 기능 : 현재 선택된 ticker에 대한 모든 interval kline을 수신한다.
        2. 매개변수
            1) max_recordes : max 1_000, 조회할 데이터의 길이를 정한다.
        """
        # 활성화된 티커가 준비될 때까지 대기
        while not self.active_tickers:
            await asyncio.sleep(
                2
            )  # utils._wait_time_sleep을 asyncio.sleep으로 대체하여 비동기 방식으로 대기

        # 모든 활성화된 티커에 대해 데이터를 수집 및 업데이트
        for ticker in self.active_tickers:
            for interval in self.KLINE_INTERVALS:
                if interval.endswith("d"):
                    limit_ = 30
                else:
                    limit_ = self.__get_kline_limit_by_days(interval=interval, days=days)
                # Kline 데이터를 수집하고 self.kline_data에 업데이트
                self.kline_data[ticker][interval] = (
                    await self.market_instance.fetch_klines_limit(
                        symbol=ticker,
                        interval=interval,
                        limit=limit_,
                    )
                )

In [33]:
def _get_kline_limit_by_days(interval: str, days: int) -> int:
    """
    1. 기능 : kline 매개변수에 들어갈 day 기간 만큼 interval의 limit값을 구한다.
    2. 매개변수
        1) interval : KLINE_INTERVALS 속성 참조
        2) day : 기간을 정한다.
    """
    day, limit = _get_valid_kline_limit(interval=interval)
    if day == 0:
        return 1000
    max_klines_per_day = (limit / day) * days

    if max_klines_per_day > 1_000:
        print(
            f"The calculated limit exceeds the maximum allowed value: {max_klines_per_day}. Limiting to 1,000."
        )
        return 1_000

    return int(max_klines_per_day)

In [34]:
_get_kline_limit_by_days(interval='5m', days=2)

576

In [35]:
def _generate_time_intervals():
    time_intervals: Dict[str, Dict] = {
        "minutes": {},  # 분 단위 타임라인
        "hours": {},  # 시간 단위 타임라인
    }

    # 각 분 초기화
    for minute in range(60):
        time_intervals["minutes"][minute] = []

    # 각 시간 초기화
    for hour in range(24):
        time_intervals["hours"][hour] = []

    # 분 단위 간격 설정
    minute_intervals = {"1m": 1, "3m": 3, "5m": 5, "15m": 15, "30m": 30}

    # 시간 단위 간격 설정
    hour_intervals = {"1h": 1, "2h": 2, "4h": 4, "6h": 6, "8h": 8, "12h": 12}

    for interval in ['5m', '15m', '1h','4h','12','1d']:
        if str(interval).endswith("m"):
            for min in range(0, 60, minute_intervals.get(interval, 0)):
                time_intervals["minutes"][min].append(interval)
        elif str(interval).endswith("h"):
            for hr in range(0, 24, hour_intervals.get(interval, 0)):
                time_intervals["hours"][hr].append(interval)
        else:
            time_intervals["hours"][0].append(interval)

    # 비어 있는 key 제거
    time_intervals["minutes"] = {
        minute: intervals
        for minute, intervals in time_intervals["minutes"].items()
        if intervals
    }
    time_intervals["hours"] = {
        hour: intervals
        for hour, intervals in time_intervals["hours"].items()
        if intervals
    }
    return time_intervals

In [36]:
_generate_time_intervals()

{'minutes': {0: ['5m', '15m'],
  5: ['5m'],
  10: ['5m'],
  15: ['5m', '15m'],
  20: ['5m'],
  25: ['5m'],
  30: ['5m', '15m'],
  35: ['5m'],
  40: ['5m'],
  45: ['5m', '15m'],
  50: ['5m'],
  55: ['5m']},
 'hours': {0: ['1h', '4h', '12', '1d'],
  1: ['1h'],
  2: ['1h'],
  3: ['1h'],
  4: ['1h', '4h'],
  5: ['1h'],
  6: ['1h'],
  7: ['1h'],
  8: ['1h', '4h'],
  9: ['1h'],
  10: ['1h'],
  11: ['1h'],
  12: ['1h', '4h'],
  13: ['1h'],
  14: ['1h'],
  15: ['1h'],
  16: ['1h', '4h'],
  17: ['1h'],
  18: ['1h'],
  19: ['1h'],
  20: ['1h', '4h'],
  21: ['1h'],
  22: ['1h'],
  23: ['1h']}}

In [1]:
time.time()

NameError: name 'time' is not defined

In [2]:
import time

In [4]:
int(time.time() * 1000)

1736574851747